In [25]:
import pandas as pd
import json
import tabulate
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show
import scipy
from sklearn.preprocessing import StandardScaler

In [2]:
output_notebook()

Loading BokehJS ...

Data Reference: https://www.kaggle.com/datasnaek/youtube-new

In [3]:
us_data = pd.read_csv('ytdata/USvideos.csv')

In [4]:
catinfo = json.load(open('ytdata/US_category_id.json'))

## Top 50 views are from these channels:

In [5]:
print(tabulate.tabulate({"Channel":us_data[us_data['category_id']==24].nlargest(50,'views')['channel_title'].unique()}, headers="keys", tablefmt="grid"))

+-----------------------------+
| Channel                     |
+=============================+
| YouTube Spotlight           |
+-----------------------------+
| Marvel Entertainment        |
+-----------------------------+
| Sony Pictures Entertainment |
+-----------------------------+
| FoxStarHindi                |
+-----------------------------+


## Likes and Dislikes Distribution

In [65]:
like_log_1 = np.log(us_data['likes'] + 1)
like_log = StandardScaler().fit_transform(like_log_1.values.reshape(-1,1))

params = scipy.stats.lognorm.fit(like_log)
params_chi = scipy.stats.chi2.fit(like_log)
params_wbmax = scipy.stats.weibull_max.fit(like_log)
print(params_wbmax)

hist, edges = np.histogram(like_log, density=True, bins=50)
mu = np.mean(like_log)
sigma = np.std(like_log)
x = np.linspace(-4,np.max(like_log),2000)
pdf_norm = scipy.stats.norm.pdf(x)
pdf_chi2 = scipy.stats.chi2.pdf(x, *params_chi)
pdf_wbmax = scipy.stats.weibull_max.pdf(x, params_wbmax[0]+1.25, *params_wbmax[1:])
pdf = scipy.stats.lognorm.pdf(x, *params)
# skpdf = skewnorm.pdf(pdf, 4)

p = figure(title="Likes Distribution", background_fill_color="#fafafa", tools='')
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
# p.line(x, pdf, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="PDF")
# p.line(x, pdf_norm+0.1, line_color="#778888", line_width=4, alpha=0.7, legend_label="PDF_NORM")
# p.line(x, pdf_chi2, line_color="#FFF999", line_width=4, alpha=0.7, legend_label="PDF_CHI2")
p.line(x, pdf_wbmax, line_color="#EEE000", line_width=4, alpha=0.7, legend_label="PDF_Weibull_Max")


dis_like_log_1 = np.log(us_data['dislikes'] + 1)
dis_like_log = StandardScaler().fit_transform(dis_like_log_1.values.reshape(-1,1))

hist_dislikes, edges_dislikes = np.histogram(dis_like_log, density=True, bins=50)
pd = figure(title="Dislikes Distribution", background_fill_color="#fafafa", tools='')
pd.quad(top=hist_dislikes, bottom=0, left=edges_dislikes[:-1], right=edges_dislikes[1:],
           fill_color="navy", line_color="white", alpha=0.5)

x = np.linspace(-4,np.max(dis_like_log),2000)
dislike_pdf_norm = scipy.stats.norm.pdf(x)
pd.line(x, dislike_pdf_norm, line_color="#EEE000", line_width=4, alpha=0.7, legend_label="PDF_Norm")

show(gridplot([p,pd], ncols=2, plot_width=600, plot_height=400, toolbar_location=None))

(2.8929950397095903, 2.8115448625071107, 3.1431575770625084)


In [9]:
from scipy.stats import skew

In [10]:
skew(hist)

1.1444402706778927

In [19]:
percentile_bins = x
percentile_cutoffs = np.percentile(like_log, x)

In [66]:
# chi_square_statistics = []
# cum_observed_frequency = np.cumsum(hist)

# dist_names = ['weibull_min', 'norm', 'weibull_max', 'beta',
#               'invgauss', 'uniform', 'gamma', 'expon',   
#               'lognorm', 'pearson3', 'triang']
# for distribution in dist_names:
#     # Set up distribution and get fitted distribution parameters
#     dist = getattr(scipy.stats, distribution)
#     param = dist.fit(like_log)
#     size = len(like_log)
#     print("{}\n{}\n".format(dist, param))


#     # Get expected counts in percentile bins
#     # cdf of fitted sistrinution across bins
#     cdf_fitted = dist.cdf(percentile_cutoffs, *param)
#     expected_frequency = []
#     for bin in range(len(percentile_bins)-1):
#         expected_cdf_area = cdf_fitted[bin+1] - cdf_fitted[bin]
#         expected_frequency.append(expected_cdf_area)

#     # Chi-square Statistics
#     expected_frequency = np.array(expected_frequency) * size
#     cum_expected_frequency = np.cumsum(expected_frequency)
#     ss = sum (((cum_expected_frequency - cum_observed_frequency) ** 2) / cum_observed_frequency)
#     chi_square_statistics.append(ss)


# #Sort by minimum ch-square statistics
# results = pd.DataFrame()
# results['Distribution'] = dist_names
# results['chi_square'] = chi_square_statistics
# results.sort_values(['chi_square'], inplace=True)


# print ('\nDistributions listed by Betterment of fit:')
# print ('............................................')
# print (results)